In [2]:
import pandas as pd
import numpy as np
from scipy import stats

## Теоретическая справка

### Критерий хи-квадрат

Используется для проверки гипотезы о независимости случайных величин. Рассмотрим основную (СВ независимы) и альтернативную (СВ зависимы) гипотезы следующих видов:

$$H_0: F_W(x, y) = F_X(x)F_Y(y) \forall x,y \in \rm I\!R$$

$$H_1: \exists x, y \in \rm I\!R: F_W(x,y)\neq F_X(x)F_Y(y)$$

#### Случай 1

$W = (X, Y)^T$ - дискретный вектор, то есть СВ $X$ принимает конечное число значений $\{a_1, a_2,\ldots, a_m\}$, а СВ $Y$ - $\{b_1, b_2,\ldots, b_k\}$

Введем обозначение $n_{ij}$ - число пар $(x, y) \in W_n = [(x_1, y_1), \ldots, (x_n, y_n)]^T$ таких, что $x = a_i, y = b_j$

$n_i = \sum\limits_{j=1}^k n_{ij}$ - число пар, где $x = a_i$, $i=\overline{1,m}$

$n_j = \sum\limits_{i=1}^m n_{ij}$ - число пар, где $y = b_j$, $j=\overline{1,k}$

$\sum\limits_{i=1}^m \sum\limits_{j=1}^k n_{ij} = n$, где $n$ - объем выборки

$p_{ij} = \frac{n_{ij}}{n}$, - вероятность встретить пару $(a_i, b_j)$

$p_{i} = \frac{n_i}{n}$, - вероятность встретить пару с $x = a_i$

$p_{j} = \frac{n_j}{n}$ - вероятность встретить пару с $y = b_j$

#### Случай 2

Если $W = (X, Y)^T$ - недискретный вектор (непрерывный), то надо сгруппировать данные (по бинам). Количество бинов выбирается произвольно. Лучше всего использовать правило Стёрджеса, то есть выбирать количество бинов ${\displaystyle q=1+\lfloor \log _{2}n\rfloor  =1+\lfloor 3.322\lg N\rfloor }$. Или более простое правило: $q \sim \sqrt n$

Тогда разбиваем область значений $X$ на $m > 1$ непересекающихся интервалов $$\{\Delta_{X, i}     \}:\bigcup\limits_{i=1}^m \Delta_{X, i} = V_X$$

Аналогично для $Y$ на $k>1$ непересекающихся интервалов ($V_Y$ - область значений СВ $Y$):

$$\{\Delta_{Y, j}     \}:\bigcup\limits_{j=1}^k \Delta_{Y, j} = V_Y$$

Тогда $n_{ij}$ - число пар $(x, y)$, таких что $x \in \Delta_{X, i}, y \in \Delta_{Y, j}$

$n_i$ - число пар, таких что $x \in \Delta_{X, i}, y$ - любой

$n_j$ - число пар, таких что $y \in \Delta_{Y, j}, x$ - любой

### Статистика

$$\tilde{\chi}^2 = T(W_n) = n \sum\limits_{i=1}^m \sum\limits_{j=1}^k \frac{(p_{ij} - p_i p_j)^2}{p_i p_j} = n \sum\limits_{i=1}^m \sum\limits_{j=1}^k \frac{(n_{ij} - \frac{n_i n_j}{n})^2}{n_i n_j}$$

Данная статистика имеет распределение хи-квадрат $\mathcal{H}_r$ с $r = (m-1)(k-1)$ степенями свободы, т.е. $\tilde{\chi}^2 \sim \mathcal{H}_r$

#### Критическая область

Пусть задан уровень значимости $\alpha$, тогда для даннного критерия критическая область имеет следующий вид:
$$
(\kappa_{1-\alpha}(r), +\infty)
$$

где $\kappa_{1-\alpha}(r)$ - квантиль уровня $1-\alpha$ распределения $\mathcal{H}_r$

#### Удобная формула для вычисления статистики

$$\tilde{\chi}^2 = n \left(\sum\limits_{i=1}^m \sum\limits_{j=1}^k \frac{(n_{ij})^2}{n_i n_j} - 1                 \right)$$

### Немного про меры связи

#### Мера связи Пирсона

$$P = \sqrt{\frac{\tilde{\chi}^2}{\tilde{\chi}^2 + n}}$$

называется коэффициентом взаимной сопряженности (или коэффициентом Пирсона)

при возрастании $m$ и $k$ стремится к коэффициенту корреляции:

$$P^2 = \frac{\tilde{\chi}^2}{\tilde{\chi}^2 + n} \rightarrow r_{XY}^2$$

Недостаток: максимальное значение $P$ равно $\sqrt{\frac{l-1}{l}} < 1$, где $l=\min(m,k)$

Поэтому Крамер ввел другую меру

#### Мера Крамера

$$C = \sqrt{\frac{\tilde{\chi}^2}{n\times \min(m-1, k-1)}}$$

называется коэффициентом Крамера. Значение лежит в отрезке $[0;1]$, максимум достигается тогда и только тогда, когда каждая строка (или столбец) содержить лишь один отличный от нуля элемент.